# Create the final consensus catalog

In [1]:
import pandas

In [2]:
id_vars = ['doid_id', 'drugbank_id']
name_vars = ['disease', 'drug']
initials = ['AJG', 'CSH', 'PK']
categories = ['DM', 'SYM', 'NOT']

## Read and process the curation results

In [3]:
curator_df = pandas.read_table('results-three-curators.tsv')
consensus_df = curator_df.copy()
consensus_df['category'] = consensus_df.PK
consensus_df['n_curators'] = consensus_df[initials].apply(lambda x: x == consensus_df['category'], axis='rows').sum(axis='columns')
consensus_df = consensus_df[id_vars + name_vars + ['category', 'n_curators']]
consensus_df.head(2)

,doid_id,drugbank_id,disease,drug,category,n_curators
0,DOID:10652,DB00245,Alzheimer's disease,Benzatropine,SYM,3
1,DOID:10652,DB00810,Alzheimer's disease,Biperiden,NOT,2


In [4]:
n_resource_df = pandas.read_table('../data/indications-slim-collapsed.tsv')
n_resource_df = n_resource_df.rename(columns={'distinct_resources': 'n_resources'})
n_resource_df = n_resource_df[['doid_id', 'drugbank_id', 'n_resources']]
consensus_df = consensus_df.merge(n_resource_df)

In [5]:
consensus_df['category'] = pandas.Categorical(consensus_df['category'], categories, ordered=True)
consensus_df = consensus_df.sort_values(['disease', 'category', 'drug'])
consensus_df.head(3)

,doid_id,drugbank_id,disease,drug,category,n_curators,n_resources
3,DOID:10652,DB00843,Alzheimer's disease,Donepezil,DM,2,1
4,DOID:10652,DB00674,Alzheimer's disease,Galantamine,DM,1,4
6,DOID:10652,DB01043,Alzheimer's disease,Memantine,DM,1,3


In [6]:
consensus_df.to_csv('../catalog/indications.tsv', index=False, sep='\t')

## Summarize by drug/disease

In [7]:
def summarize(consensus_df, index_columns):
    df = consensus_df.groupby(index_columns + ['category']).apply(len).reset_index()
    df = pandas.pivot_table(df, values=0, index=index_columns, columns='category', fill_value=0)
    df = df.reset_index()[index_columns + categories]
    df['total'] = df[categories].sum(axis='columns')
    df = df.sort_values(index_columns[1])
    return df

In [8]:
disease_summary_df = summarize(consensus_df, ['doid_id', 'disease'])
disease_summary_df.to_csv('../catalog/diseases.tsv', index=False, sep='\t')
disease_summary_df.head(2)

category,doid_id,disease,DM,SYM,NOT,total
8,DOID:10652,Alzheimer's disease,4,5,7,16
91,DOID:9206,Barrett's esophagus,2,0,0,2


In [9]:
# Number of diseases with one or more indication by category
(disease_summary_df[categories] > 0).sum()

category
DM     77
SYM    50
NOT    53
dtype: int64

In [10]:
drug_summary_df = summarize(consensus_df, ['drugbank_id', 'drug'])
drug_summary_df.to_csv('../catalog/drugs.tsv', index=False, sep='\t')
drug_summary_df.head(2)

category,drugbank_id,drug,DM,SYM,NOT,total
432,DB01048,Abacavir,1,0,0,1
575,DB05812,Abiraterone,1,0,0,1


In [11]:
# Number of drugs with one or more indication by category
(drug_summary_df[categories] > 0).sum()

category
DM     387
SYM    221
NOT    157
dtype: int64

## Export to excel

In [12]:
writer = pandas.ExcelWriter('../catalog/catalog.xlsx')
consensus_df.to_excel(writer, sheet_name='indications', index=False)
disease_summary_df.to_excel(writer, sheet_name='diseases', index=False)
drug_summary_df.to_excel(writer, sheet_name='drugs', index=False)
curator_df[id_vars + name_vars + initials].to_excel(writer, sheet_name='curation', index=False)
for sheet in writer.sheets.values():
    sheet.freeze_panes(1, 0)
writer.close()

## Crostabulations

In [13]:
def crosstab(df, rows, cols='category'):
    count_df = pandas.crosstab(df[rows], df[cols])
    count_df = count_df[categories]
    totals = count_df.sum(axis='columns')
    percent_df = 100 * count_df.divide(totals, axis='rows')
    count_df['Total'] = totals
    return count_df, percent_df

### Category breakdown by number of resources

In [14]:
count_df, percent_df = crosstab(consensus_df, 'n_resources', 'category')

In [15]:
count_df

category,DM,SYM,NOT,Total
n_resources,,,,
1,433,271,210,914
2,190,74,23,287
3,75,38,10,123
4,57,7,0,64


In [16]:
percent_df.round(1)

,DM,SYM,NOT
n_resources,,,
1,47.4,29.6,23.0
2,66.2,25.8,8.0
3,61.0,30.9,8.1
4,89.1,10.9,0.0


### Category breakdown by resource

In [17]:
source_df = pandas.read_table('../data/indications-slim-verbose.tsv')
source_df = source_df.rename(columns={'do_slim_id': 'doid_id'})
source_df = source_df[id_vars + ['resource']].drop_duplicates()
source_df = source_df.query("resource != 'medi_lps'")
source_df = consensus_df.merge(source_df)
source_df.head(2)

,doid_id,drugbank_id,disease,drug,category,n_curators,n_resources,resource
0,DOID:10652,DB00843,Alzheimer's disease,Donepezil,DM,2,1,predict
1,DOID:10652,DB00674,Alzheimer's disease,Galantamine,DM,1,4,ehrlink


In [18]:
count_df, percent_df = crosstab(source_df, 'resource', 'category')

In [19]:
count_df

category,DM,SYM,NOT,Total
resource,,,,
ehrlink,205,163,95,463
labeledin,183,72,22,277
medi_hps,532,168,93,793
predict,346,158,76,580


In [20]:
percent_df.round(1)

,DM,SYM,NOT
resource,,,
ehrlink,44.3,35.2,20.5
labeledin,66.1,26.0,7.9
medi_hps,67.1,21.2,11.7
predict,59.7,27.2,13.1
